#Read in Data

In [1]:
import pandas as pd
x_train = pd.read_csv('X_train.csv')
x_test = pd.read_csv('X_val.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_val.csv')

#Model definitions

In [ ]:
#Installation
!pip install interpret -q

In [4]:
from pandas.core.window.rolling import numba_notes
#Regression Models
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor as XGBRegressor
#Linear Regression
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
#Ridge Regression
from sklearn import linear_model
ridge = linear_model.Ridge(alpha=.99)
#Lasso Regression
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
lasso = linear_model.Lasso(alpha=0.1)
#KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3, weights='distance')
#Random Forrest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=30, min_samples_leaf=4)
#Decision Trees
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=3, min_samples_leaf=3, min_weight_fraction_leaf=0.2)
#SVM - SVR
from sklearn import svm
SVM = svm.SVR()
#Gradient Boosting Regressor
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(learning_rate = 0.02, n_estimators = 80, random_state=0)
# XGBoost
import xgboost as xg
xgb = xg.XGBRegressor(learning_rate = 0.1, max_depth = 10, n_estimators = 100)
# EBM
from interpret import show
from interpret.data import Marginal
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree
ebm = ExplainableBoostingRegressor(interactions=100)

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
x_train_here = x_train.values
y_train_here = y_train.values

In [6]:
models = [lasso,  rf, dt, SVM, gbr, xgb, ebm] #lm, ridge, neigh
labels = ['Lasso', 'Random Forrest', 'Deision Tree', 'SVM', 'Gradient Boosting', 'XGBoost', 'EBM'] # 'Linear', 'Ridge', 'KNN'

In [7]:
models

[Lasso(alpha=0.1),
 RandomForestRegressor(max_depth=30, min_samples_leaf=4),
 DecisionTreeRegressor(max_depth=3, min_samples_leaf=3,
                       min_weight_fraction_leaf=0.2),
 SVR(),
 GradientBoostingRegressor(learning_rate=0.02, n_estimators=80, random_state=0),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=

New

In [27]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression

# Reduce the number of base models
reduced_models = [lasso, rf, xgb]  # Select a subset of the original models

simple_meta_regressor = LinearRegression()

stack = StackingCVRegressor(regressors=reduced_models,
                            meta_regressor=simple_meta_regressor,  # Use scikit-learn's LinearRegression
                            cv=2,
                            use_features_in_secondary=False,
                            store_train_meta_features=True,
                            shuffle=False
                           )

In [28]:
y_train_here = y_train_here.ravel()
stack.fit(x_train_here, y_train_here)
pred = stack.predict(x_test)
score_mae = mean_absolute_error(y_test, pred)
score_mse = mean_squared_error(y_test, pred)
score_r2=r2_score(y_test, pred)*100
score_rmse =  sqrt(score_mse)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [29]:
stacking_results = []
stacking_results.append({'Model': 'Stacked Model', 'Mean Absolute Error': score_mae, 'Mean Squared Error': score_mse, 'RMSE': score_rmse,'R-squared': score_r2})

df = pd.DataFrame(stacking_results)
df

,Model,Mean Absolute Error,Mean Squared Error,RMSE,R-squared
0,Stacked Model,0.011888,0.000384,0.0196,96.240089
